In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
bs = 32 # batch size
version = 0

In [3]:
from fastai import *
from fastai.vision.all import *
from fastai.vision.widgets import *

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
data_path = "data"
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files,                        # get_image_files returns a list of all images in that path recursively by default
    splitter=RandomSplitter(valid_pct=0.1, seed=42),  # getting 90-10 train-validation split
    get_y=parent_label,                               # taking the folder name as labels
    item_tfms=Resize(128))                            # resizing to get the image of same shape

In [ ]:
dblock = dblock.new(item_tfms=RandomResizedCrop(224, min_scale=0.5), batch_tfms=aug_transforms())
dls = dblock.dataloaders(data_path, bs = bs)
torch.save(dls, f"dataloaders/cap_dataloader_v{version}.pkl")

In [ ]:
# dls = torch.load(f"dataloaders/cap_dataloader_v{version}.pkl")

In [ ]:
dls.train.show_batch(max_n=8, nrows=2)

In [ ]:
dls.valid.show_batch(max_n=8, nrows=2)

# Training and Data Cleaning

In [ ]:
model_path = "models"

In [ ]:
model = vision_learner(dls, resnet34, metrics=[error_rate,accuracy])

In [ ]:
#model.load(f"cap-recognizer-v{version-1}")

In [ ]:
model.fine_tune(5)

In [ ]:
model.fine_tune(1)

In [ ]:
model.save(f"cap-recognizer-v{version}")

In [ ]:
interp = ClassificationInterpretation.from_learner(model)
interp.plot_confusion_matrix()